<a href="https://colab.research.google.com/github/pycriador/databricks-api/blob/main/databricks_API_Download_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lista de arquivos via databricks API

Script simples para fazer a listagem de todos CSV do DBFS do DataBricks e fazer download dos arquivos

Instalando depedências:

In [ ]:
!pip install requests
!pip install databricks-cli

Criar token e informar URL do servidor e pastas desejadas.

Veja como criar Token pela interface do databricks [DOC](https://docs.databricks.com/dev-tools/api/latest/authentication.html)

In [ ]:
import requests
import json
from databricks_cli.sdk.api_client import ApiClient
from databricks_cli.dbfs.api import DbfsApi
from databricks_cli.dbfs.dbfs_path import DbfsPath
import re
from datetime import datetime
import math

#Função para converter o tamanho do arquivo em um melhor formato
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

#Coloque aqui o Token e a URL do seu servidor databricks
token = ''
url=''


api_client = ApiClient(
  host  = url,
  token = token
)

#URL API para o dbfs list
api_url=f'{url}/api/2.0/dbfs/list'
headers= { "Authorization" : f"Bearer {token}" }

path_list = ['/FileStore/pasta1/', '/FileStore/pasta2/', '/FileStore/pasta3/']

for path in path_list:
  payload= { "path": path }
  response = requests.request("GET", api_url, json=payload, headers=headers)

  #print(response.text)
  result = json.loads(response.text)
  for info in result['files']:
    #Filtrar somente os arquivos CSV
    if re.search('.*.csv', str(info['path'])):
      print("=======================================")
      print(f"Arquivo: {info['path']}")
      print(f"Tamanho: {convert_size(info['file_size'])}")
      
      #Coletar data de modificação
      data =  info['modification_time']
      ts = int(str(data)[:10])
      #Mostrar na tela a hora de modificação
      modificacao = datetime.utcfromtimestamp(ts).strftime('%d-%m-%Y %H:%M:%S')
      print(f"Modificado em: {modificacao}")
      dbfs_source_file_path      = f'dbfs:{info["path"]}'
      local_file_download_path   = f'{path.split("/")[-2]}.csv'

      dbfs_api  = DbfsApi(api_client)
      dbfs_path = DbfsPath(dbfs_source_file_path)

      # Download local
      dbfs_api.get_file(dbfs_path, local_file_download_path,overwrite = True)